# Create a cube using PyVista

This notebook creates a cube using PyVista. There are 3 different iterations:

1. Use PyVista's Delaunay triangulation method to create a cube with 8 vertices.
2. Construct a cube with 8 vertices from scratch (ish) by generating arrays for vertices and vertex normals, and using `scipy.spatial.ConvexHull` to generate a triangles array.
3. Construct a cube with a variable number of vertices per face from scratch (ish) by generating arrays for vertices and vertex normals, and using `scipy.spatial.ConvexHull` to generate a triangles array.


In [2]:
import numpy as np
import pyvista as pv
from scipy.spatial import ConvexHull

## 1 - Generate a cube with 8 vertices using PyVista Delaunay triangulation


### 1.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of $x$, $y$, $z$ values.


In [2]:
x = np.linspace(0, 1, 2)
y = np.linspace(0, 1, 2)
z = np.linspace(0, 1, 2)

Combine the $x$, $y$, $z$ values into an array, `vertices`, with the correct shape. A cube has 8 vertices, and each vertex has 3 coordinates, so `vertices` should have shape (8,3)


In [3]:
x_v, y_v, z_v = np.meshgrid(x, y, z)
vertices = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

vertices.shape

(8, 3)

### 1.2 - PyVista mesh

We want to create a PyVista mesh.


Create and plot the PyVista mesh. This mesh only has vertices (no faces).


In [4]:
mesh = pv.PolyData(vertices)
mesh.plot()

Widget(value='<iframe src="http://localhost:56109/index.html?ui=P_0x1287db4d0_0&reconnect=auto" class="pyvista…

### 1.3 - Connected surface

We want to create a connected surface.


Create and plot a connected surface, using Delaunay triangulation.


In [5]:
surface = mesh.delaunay_3d()
surface.plot(show_edges=True)

Widget(value='<iframe src="http://localhost:56109/index.html?ui=P_0x154554f50_1&reconnect=auto" class="pyvista…

## 2 - Generate a cube with 8 vertices from scratch (ish)


### 2.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of $x$, $y$, $z$ values.


In [6]:
x = np.linspace(0, 1, 2)
y = np.linspace(0, 1, 2)
z = np.linspace(0, 1, 2)

Combine the $x$, $y$, $z$ values into an array, `vertices`, with the correct shape. A cube has 8 vertices, and each vertex has 3 coordinates, so `vertices` should have shape (8,3)


In [7]:
x_v, y_v, z_v = np.meshgrid(x, y, z)
vertices = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

vertices.shape

(8, 3)

### 2.2 - Vertex normals

We want to generate an array of vertex normals with the correct shape.


Calculate the normals.


In [8]:
center = np.array([0.5, 0.5, 0.5])
normals = vertices - center[None, :]

Normalize the normals. Similarly to `vertices`, `normals` should have shape (8,3).


In [9]:
normals = normals / np.linalg.norm(normals, axis=1)[:, None]

normals.shape

(8, 3)

### 2.3 - Triangles

We want to generate an array of triangles.


The triangulation is handled by SciPy's `ConvexHull` method. The triangle array, `triangles`, should have shape (12, 3) - there are 12 triangles (2 triangles per face), and each triangle has 3 vertices. The format of `triangles` is `[[v1, v2, v3], [v4, v5, v6], ...]`.


In [10]:
triangles = ConvexHull(vertices).simplices

triangles.shape

(12, 3)

### 2.4 - PyVista mesh

We want to create the PyVista mesh.


PyVista expects the triangles to be given in the format `[3, v1, v2, v3, 3, v4, v5, v6, ...]`. Reformat `triangles`.


In [11]:
triangles_reformatted = np.zeros((triangles.shape[0], 4), dtype=int)
triangles_reformatted[:, 0] = 3
triangles_reformatted[:, 1:] = triangles

Flatten `triangles_reformatted`. The flattened array, `triangles_flattened`, should have shape (48,) - there are 12 triangles, each with 3 vertices; 12 $\cdot$ (3+1) = 48.


In [12]:
triangles_flattened = triangles_reformatted.ravel()

triangles_flattened.shape

(48,)

Create the PyVista mesh, and add the normals as point data. The mesh should have 8 vertices, and 12 cells.


In [13]:
cube_mesh = pv.PolyData(vertices, triangles_flattened)
cube_mesh.point_data["normals"] = normals

print(f"Number of vertices: {cube_mesh.n_points}")
print(f"Number of cells: {cube_mesh.n_cells}")

Number of vertices: 8
Number of cells: 12


### 2.5 - Plot the mesh

We want to plot the PyVista mesh.


Plot the PyVista mesh, along with the vertex normals.


In [14]:
pl = pv.Plotter()

pl.add_mesh(cube_mesh, show_edges=True, color="lightblue")
arrows = cube_mesh.glyph(orient="normals", scale=False, factor=0.3)
pl.add_mesh(arrows, color="red")

pl.show()

Widget(value='<iframe src="http://localhost:56109/index.html?ui=P_0x162660190_2&reconnect=auto" class="pyvista…

## 3 - Generate a cube with a variable number of vertices per face from scratch (ish)


### 3.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of all possible $x$, $y$, $z$ values. $N$ is the number of vertices per edge.


In [107]:
N = 11

x = np.linspace(0, 1, N)
y = np.linspace(0, 1, N)
z = np.linspace(0, 1, N)

Generate vertices for each plane. Each array should have shape ($N^2$, 3)


In [108]:
# x = 0 plane.
x_v, y_v, z_v = np.meshgrid(np.array([0]), y, z)
vertices_x_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# x = 1 plane.
x_v, y_v, z_v = np.meshgrid(np.array([1]), y, z)
vertices_x_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# y = 0 plane.
x_v, y_v, z_v = np.meshgrid(x, np.array([0]), z)
vertices_y_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# y = 1 plane.
x_v, y_v, z_v = np.meshgrid(x, np.array([1]), z)
vertices_y_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# z = 0 plane.
x_v, y_v, z_v = np.meshgrid(x, y, np.array([0]))
vertices_z_0 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

# z = 1 plane.
x_v, y_v, z_v = np.meshgrid(x, y, np.array([1]))
vertices_z_1 = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

vertices_z_1.shape

(121, 3)

Combine the all of the vertices into one array, `vertices`. `vertices` should have shape ($6 N^2$, 3).


In [109]:
# Number of vertices per face.
num_vertices = vertices_z_1.shape[0]

# Initialize an empty array to store the vertices.
vertices = np.zeros((num_vertices * 6, 3), dtype=float)

# Populate the array.
for i, arr in enumerate(
    [vertices_x_0, vertices_x_1, vertices_y_0, vertices_y_1, vertices_z_0, vertices_z_1]
):
    vertices[i * num_vertices : (i + 1) * num_vertices, :] = arr

vertices.shape

(726, 3)

Remove duplicate elements from `vertices`. This block of code may need to be modified to account for floating point errors.


In [110]:
vertices = np.unique(vertices, axis=0)
vertices.shape

(602, 3)

### 3.2 - Vertex normals

We want to generate an array of vertex normals with the correct shape.


Calculate the normals.


In [111]:
center = np.array([0.5, 0.5, 0.5])
normals = vertices - center[None, :]

Normalize the normals. `normals` should have the same shape as `vertices`.


In [112]:
normals = normals / np.linalg.norm(normals, axis=1)[:, None]

normals.shape

(602, 3)

### 3.3 - Triangles

We want to generate an array of triangles.


The triangulation is handled by SciPy's `ConvexHull` method. The format of the triangles array, `triangles`, is `[[v1, v2, v3], [v4, v5, v6], ...]`.


In [113]:
triangles = ConvexHull(vertices).simplices

triangles.shape

(12, 3)

### 3.4 - PyVista mesh

We want to create the PyVista mesh.


PyVista expects the triangles to be given in the format `[3, v1, v2, v3, 3, v4, v5, v6, ...]`. Reformat `triangles`.


In [114]:
triangles_reformatted = np.zeros((triangles.shape[0], 4), dtype=int)
triangles_reformatted[:, 0] = 3
triangles_reformatted[:, 1:] = triangles

Flatten `triangles_reformatted` to a 1D array, `triangles_flattened`.


In [115]:
triangles_flattened = triangles_reformatted.ravel()

triangles_flattened.shape

(48,)

Create the PyVista mesh, and add the normals as point data.


In [116]:
cube_mesh = pv.PolyData(vertices, triangles_flattened)
cube_mesh.point_data["normals"] = normals

print(f"Number of vertices: {cube_mesh.n_points}")
print(f"Number of cells: {cube_mesh.n_cells}")

Number of vertices: 602
Number of cells: 12


### 3.5 - Plot the mesh

We want to plot the PyVista mesh.


Plot the PyVista mesh.


In [118]:
pl = pv.Plotter()

pl.add_mesh(cube_mesh, show_edges=True, color="lightblue")

arrows = cube_mesh.glyph(orient="normals", scale=False, factor=0.3)
pl.add_mesh(arrows, color="red")

pl.show()

Widget(value='<iframe src="http://localhost:57638/index.html?ui=P_0x3127a6fd0_9&reconnect=auto" class="pyvista…